In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
from keras.layers import LSTM, Dense, SimpleRNN
import tensorflow as tf
import statistics
import keras
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
%load_ext tensorboard

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [4]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent'

In [5]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [6]:
d = pd.read_csv("../mapped_data/xalan-2.4.csv")
d['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [7]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [8]:
d

data_path   bugs
0    ../dataset/promise/defect data/xalan-2.4/xalan...      0
1    ../dataset/promise/defect data/xalan-2.4/xalan...      2
2    ../dataset/promise/defect data/xalan-2.4/xalan...      0
3    ../dataset/promise/defect data/xalan-2.4/xalan...      0
4    ../dataset/promise/defect data/xalan-2.4/xalan...      0
..                                                 ...    ...
671  ../dataset/promise/defect data/xalan-2.4/xalan...      0
672  ../dataset/promise/defect data/xalan-2.4/xalan...      0
673  ../dataset/promise/defect data/xalan-2.4/xalan...      0
674  ../dataset/promise/defect data/xalan-2.4/xalan...      0
675  ../dataset/promise/defect data/xalan-2.4/xalan...      0

[676 rows x 2 columns]

In [9]:
train, test = train_test_split(d, test_size=0.2)

In [10]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [11]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [12]:
train_x[0]

[2325.0,
 2325.0,
 8525.0,
 8860.0,
 1019.0,
 5610.0,
 5129.0,
 3522.0,
 1019.0,
 4407.0,
 1019.0,
 4704.0,
 5129.0,
 5610.0,
 4407.0,
 3522.0,
 7911.0,
 5129.0,
 8860.0,
 3522.0,
 5756.0,
 4406.0,
 5362.0,
 5007.0,
 5756.0,
 7253.0,
 5362.0,
 5007.0,
 5610.0,
 7253.0,
 4406.0,
 3522.0,
 7911.0,
 5129.0,
 8860.0,
 5610.0,
 7253.0,
 4406.0,
 4327.0,
 703.0,
 1806.0,
 5129.0,
 2568.0,
 7013.0,
 5100.0,
 5610.0,
 7013.0,
 3522.0,
 1063.0,
 7005.0,
 4943.0,
 5610.0,
 7005.0,
 2240.0,
 3522.0,
 5750.0,
 2240.0,
 7005.0,
 8860.0,
 5610.0,
 7005.0,
 2430.0,
 3522.0,
 2430.0,
 2430.0,
 7005.0,
 2240.0,
 7674.0,
 5233.0,
 5756.0,
 7118.0,
 6076.0,
 4613.0,
 4615.0,
 5756.0,
 5594.0,
 5594.0,
 7118.0,
 5594.0,
 3522.0,
 5750.0,
 7674.0,
 5594.0,
 8860.0,
 3522.0,
 3529.0,
 344.0,
 4193.0,
 1044.0,
 3421.0,
 4258.0,
 9271.0,
 5532.0,
 3522.0,
 2326.0,
 5571.0,
 2326.0,
 7301.0,
 9271.0,
 5571.0,
 3522.0,
 7822.0,
 5385.0,
 5230.0,
 5571.0,
 5153.0,
 7833.0,
 2326.0,
 5571.0,
 6949.0,
 7822.0,
 92

In [13]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

9459.0

In [14]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [15]:
train_y

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 7, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 0, 0, 3, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [17]:
train_x.shape

(540, 7367)

In [18]:
def define_model():
    # define the model
    model = Sequential()
    model.add(tf.keras.layers.Embedding(10671, 6, input_length=train_x.shape[1]))
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    # compile the model
    model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )  
    return model

In [19]:
y = list(test_y)
y

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0]

In [20]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
total_predictions = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/AST_Training_curves/5dim_xalan2.4/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [round(i) for i in x]
    total_predictions.append(x)
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
6/6 [==============================] - 1s 46ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 2/100
6/6 [==============================] - 0s 40ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 3/100
6/6 [==============================] - 0s 39ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 4/100
6/6 [==============================] - 0s 41ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 5/100
6/6 [==============================] - 0s 39ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 6/100
6/6 [==============================] - 0s 39ms/step - loss: 0.4056 -

6/6 [==============================] - 0s 72ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 89/100
6/6 [==============================] - 0s 78ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 90/100
6/6 [==============================] - 1s 97ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 91/100
6/6 [==============================] - 1s 114ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 92/100
6/6 [==============================] - 1s 106ms/step - loss: 0.4056 - mse: 0.4056 - mae: 0.2130 - root_mean_squared_error: 0.6368 - mean_squared_logarithmic_error: 0.1158
Epoch 93/100
6/6 [==============================] - 1s 103ms/step - loss: 0.4056 - mse: 0.4056 -

6/6 [==============================] - 1s 94ms/step - loss: 0.1408 - mse: 0.1408 - mae: 0.2253 - root_mean_squared_error: 0.3752 - mean_squared_logarithmic_error: 0.0505
Epoch 75/100
6/6 [==============================] - 1s 98ms/step - loss: 0.1387 - mse: 0.1387 - mae: 0.2241 - root_mean_squared_error: 0.3725 - mean_squared_logarithmic_error: 0.0501
Epoch 76/100
6/6 [==============================] - 1s 94ms/step - loss: 0.1370 - mse: 0.1370 - mae: 0.2226 - root_mean_squared_error: 0.3702 - mean_squared_logarithmic_error: 0.0495
Epoch 77/100
6/6 [==============================] - 1s 105ms/step - loss: 0.1352 - mse: 0.1352 - mae: 0.2214 - root_mean_squared_error: 0.3677 - mean_squared_logarithmic_error: 0.0491
Epoch 78/100
6/6 [==============================] - 1s 99ms/step - loss: 0.1337 - mse: 0.1337 - mae: 0.2204 - root_mean_squared_error: 0.3657 - mean_squared_logarithmic_error: 0.0488
Epoch 79/100
6/6 [==============================] - 1s 101ms/step - loss: 0.1321 - mse: 0.1321 - 

6/6 [==============================] - 1s 103ms/step - loss: 0.1843 - mse: 0.1843 - mae: 0.2099 - root_mean_squared_error: 0.4293 - mean_squared_logarithmic_error: 0.0569
Epoch 61/100
6/6 [==============================] - 1s 98ms/step - loss: 0.1821 - mse: 0.1821 - mae: 0.2125 - root_mean_squared_error: 0.4268 - mean_squared_logarithmic_error: 0.0571
Epoch 62/100
6/6 [==============================] - 1s 104ms/step - loss: 0.1796 - mse: 0.1796 - mae: 0.2137 - root_mean_squared_error: 0.4238 - mean_squared_logarithmic_error: 0.0569
Epoch 63/100
6/6 [==============================] - 1s 97ms/step - loss: 0.1787 - mse: 0.1787 - mae: 0.1979 - root_mean_squared_error: 0.4227 - mean_squared_logarithmic_error: 0.0548
Epoch 64/100
6/6 [==============================] - 1s 99ms/step - loss: 0.1750 - mse: 0.1750 - mae: 0.1930 - root_mean_squared_error: 0.4183 - mean_squared_logarithmic_error: 0.0533
Epoch 65/100
6/6 [==============================] - 1s 105ms/step - loss: 0.1746 - mse: 0.1746 -

6/6 [==============================] - 1s 95ms/step - loss: 0.2490 - mse: 0.2490 - mae: 0.2869 - root_mean_squared_error: 0.4990 - mean_squared_logarithmic_error: 0.0761
Epoch 47/100
6/6 [==============================] - 1s 106ms/step - loss: 0.2467 - mse: 0.2467 - mae: 0.2242 - root_mean_squared_error: 0.4967 - mean_squared_logarithmic_error: 0.0684
Epoch 48/100
6/6 [==============================] - 1s 105ms/step - loss: 0.2442 - mse: 0.2442 - mae: 0.2381 - root_mean_squared_error: 0.4941 - mean_squared_logarithmic_error: 0.0692
Epoch 49/100
6/6 [==============================] - 1s 111ms/step - loss: 0.2426 - mse: 0.2426 - mae: 0.2751 - root_mean_squared_error: 0.4926 - mean_squared_logarithmic_error: 0.0732
Epoch 50/100
6/6 [==============================] - 1s 109ms/step - loss: 0.2428 - mse: 0.2428 - mae: 0.2201 - root_mean_squared_error: 0.4927 - mean_squared_logarithmic_error: 0.0668
Epoch 51/100
6/6 [==============================] - 1s 97ms/step - loss: 0.2402 - mse: 0.2402 

...

6/6 [==============================] - 0s 63ms/step - loss: 0.2495 - mse: 0.2495 - mae: 0.2887 - root_mean_squared_error: 0.4995 - mean_squared_logarithmic_error: 0.0708
Epoch 34/100
6/6 [==============================] - 0s 42ms/step - loss: 0.2449 - mse: 0.2449 - mae: 0.2800 - root_mean_squared_error: 0.4949 - mean_squared_logarithmic_error: 0.0690
Epoch 35/100
6/6 [==============================] - 0s 37ms/step - loss: 0.2424 - mse: 0.2424 - mae: 0.2578 - root_mean_squared_error: 0.4924 - mean_squared_logarithmic_error: 0.0661
Epoch 36/100
6/6 [==============================] - 0s 37ms/step - loss: 0.2400 - mse: 0.2400 - mae: 0.2555 - root_mean_squared_error: 0.4898 - mean_squared_logarithmic_error: 0.0656
Epoch 37/100
6/6 [==============================] - 0s 37ms/step - loss: 0.2388 - mse: 0.2388 - mae: 0.2757 - root_mean_squared_error: 0.4887 - mean_squared_logarithmic_error: 0.0683
Epoch 38/100
6/6 [==============================] - 0s 37ms/step - loss: 0.2336 - mse: 0.2336 - ma

6/6 [==============================] - 1s 100ms/step - loss: 0.3300 - mse: 0.3300 - mae: 0.3332 - root_mean_squared_error: 0.5745 - mean_squared_logarithmic_error: 0.0907
Epoch 20/100
6/6 [==============================] - 1s 95ms/step - loss: 0.3258 - mse: 0.3258 - mae: 0.3381 - root_mean_squared_error: 0.5708 - mean_squared_logarithmic_error: 0.0901
Epoch 21/100
6/6 [==============================] - 1s 106ms/step - loss: 0.3205 - mse: 0.3205 - mae: 0.3418 - root_mean_squared_error: 0.5661 - mean_squared_logarithmic_error: 0.0894
Epoch 22/100
6/6 [==============================] - 1s 96ms/step - loss: 0.3164 - mse: 0.3164 - mae: 0.3155 - root_mean_squared_error: 0.5625 - mean_squared_logarithmic_error: 0.0842
Epoch 23/100
6/6 [==============================] - 1s 100ms/step - loss: 0.3117 - mse: 0.3117 - mae: 0.3261 - root_mean_squared_error: 0.5583 - mean_squared_logarithmic_error: 0.0846
Epoch 24/100
6/6 [==============================] - 1s 96ms/step - loss: 0.3075 - mse: 0.3075 -

6/6 [==============================] - 0s 63ms/step - loss: 0.3817 - mse: 0.3817 - mae: 0.2198 - root_mean_squared_error: 0.6178 - mean_squared_logarithmic_error: 0.1036
Epoch 6/100
6/6 [==============================] - 0s 63ms/step - loss: 0.3751 - mse: 0.3751 - mae: 0.2210 - root_mean_squared_error: 0.6125 - mean_squared_logarithmic_error: 0.1007
Epoch 7/100
6/6 [==============================] - 0s 65ms/step - loss: 0.3686 - mse: 0.3686 - mae: 0.2222 - root_mean_squared_error: 0.6071 - mean_squared_logarithmic_error: 0.0978
Epoch 8/100
6/6 [==============================] - 0s 73ms/step - loss: 0.3623 - mse: 0.3623 - mae: 0.2236 - root_mean_squared_error: 0.6019 - mean_squared_logarithmic_error: 0.0951
Epoch 9/100
6/6 [==============================] - 0s 64ms/step - loss: 0.3548 - mse: 0.3548 - mae: 0.2253 - root_mean_squared_error: 0.5956 - mean_squared_logarithmic_error: 0.0923
Epoch 10/100
6/6 [==============================] - 0s 70ms/step - loss: 0.3479 - mse: 0.3479 - mae: 0

6/6 [==============================] - 1s 96ms/step - loss: 0.1115 - mse: 0.1115 - mae: 0.2011 - root_mean_squared_error: 0.3339 - mean_squared_logarithmic_error: 0.0431
Epoch 94/100
6/6 [==============================] - 1s 104ms/step - loss: 0.1102 - mse: 0.1102 - mae: 0.1997 - root_mean_squared_error: 0.3319 - mean_squared_logarithmic_error: 0.0427
Epoch 95/100
6/6 [==============================] - 1s 111ms/step - loss: 0.1089 - mse: 0.1089 - mae: 0.1986 - root_mean_squared_error: 0.3300 - mean_squared_logarithmic_error: 0.0423
Epoch 96/100
6/6 [==============================] - 1s 95ms/step - loss: 0.1077 - mse: 0.1077 - mae: 0.1974 - root_mean_squared_error: 0.3282 - mean_squared_logarithmic_error: 0.0419
Epoch 97/100
6/6 [==============================] - 1s 105ms/step - loss: 0.1065 - mse: 0.1065 - mae: 0.1965 - root_mean_squared_error: 0.3264 - mean_squared_logarithmic_error: 0.0417
Epoch 98/100
6/6 [==============================] - 1s 92ms/step - loss: 0.1051 - mse: 0.1051 -

6/6 [==============================] - 1s 97ms/step - loss: 0.1111 - mse: 0.1111 - mae: 0.1519 - root_mean_squared_error: 0.3334 - mean_squared_logarithmic_error: 0.0375
Epoch 79/100
6/6 [==============================] - 1s 95ms/step - loss: 0.1085 - mse: 0.1085 - mae: 0.1531 - root_mean_squared_error: 0.3294 - mean_squared_logarithmic_error: 0.0373
Epoch 80/100
6/6 [==============================] - 1s 106ms/step - loss: 0.1070 - mse: 0.1070 - mae: 0.1572 - root_mean_squared_error: 0.3271 - mean_squared_logarithmic_error: 0.0378
Epoch 81/100
6/6 [==============================] - 1s 98ms/step - loss: 0.1047 - mse: 0.1047 - mae: 0.1505 - root_mean_squared_error: 0.3236 - mean_squared_logarithmic_error: 0.0365
Epoch 82/100
6/6 [==============================] - 1s 106ms/step - loss: 0.1024 - mse: 0.1024 - mae: 0.1459 - root_mean_squared_error: 0.3201 - mean_squared_logarithmic_error: 0.0354
Epoch 83/100
6/6 [==============================] - 1s 104ms/step - loss: 0.1001 - mse: 0.1001 -

In [21]:
%tensorboard --logdir '/home/bavanya/Documents/AST_Training_curves/5dim_xalan2.4/'

In [22]:
total_training_time

[41.4630606174469,
 62.277870416641235,
 82.78148603439331,
 64.61135268211365,
 60.43436288833618,
 56.723592042922974,
 82.86562037467957,
 65.37496709823608,
 61.85675573348999,
 66.08905911445618,
 61.96762752532959,
 82.85429382324219,
 29.86557674407959,
 56.942795753479004,
 62.833099126815796,
 49.91772651672363,
 62.124128341674805,
 53.009713649749756,
 50.30627965927124,
 82.87302112579346,
 82.89161252975464,
 50.941336154937744,
 50.946460485458374,
 47.3077187538147,
 51.39426350593567,
 62.62112784385681,
 49.817718744277954,
 59.633838176727295,
 58.22213792800903,
 56.346176624298096]

In [23]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 60.243159333864845


In [24]:
total_training_loss_history

[{'loss': [0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   0.4055555462837219,
   

In [25]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.13124191984534264
avg mae of training: 0.1717672700683276


In [26]:
training_mse

[0.4055555462837219,
 0.10130957514047623,
 0.09928365051746368,
 0.14745812118053436,
 0.11710884422063828,
 0.11372818797826767,
 0.4055555462837219,
 0.08829611539840698,
 0.12205792218446732,
 0.12468025833368301,
 0.4055555462837219,
 0.09178312867879868,
 0.09776131063699722,
 0.09605444222688675,
 0.08540176600217819,
 0.10098300129175186,
 0.07140590250492096,
 0.09752565622329712,
 0.11598317325115204,
 0.08727956563234329,
 0.10096118599176407,
 0.0949578806757927,
 0.0917624905705452,
 0.08884128928184509,
 0.1018734723329544,
 0.09933415055274963,
 0.09322652965784073,
 0.11776549369096756,
 0.10262414813041687,
 0.07114369422197342]

In [27]:
training_mae

[0.21296297013759613,
 0.1937977820634842,
 0.1438366323709488,
 0.19624602794647217,
 0.20537292957305908,
 0.16979379951953888,
 0.21296297013759613,
 0.17416512966156006,
 0.19725340604782104,
 0.20652849972248077,
 0.21296297013759613,
 0.13354085385799408,
 0.18506254255771637,
 0.17680127918720245,
 0.13398681581020355,
 0.14657841622829437,
 0.12114416062831879,
 0.18051785230636597,
 0.18523651361465454,
 0.15286757051944733,
 0.1711575835943222,
 0.16915039718151093,
 0.1825825721025467,
 0.14420080184936523,
 0.14445175230503082,
 0.18403351306915283,
 0.16040873527526855,
 0.14587338268756866,
 0.1934836357831955,
 0.11605660617351532]

In [28]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse of testing: " + str(sum_mse/30))
print("avg mae of testing: " + str(sum_mae/30))
print("avg are of testing: " + str(sum_are/30))

avg mse of testing: 0.5258203307787578
avg mae of testing: 0.3890054613351822
avg are of testing: 0.1722834974527359


In [29]:
test_mse

[0.6102941036224365,
 0.49952900409698486,
 0.5408803820610046,
 0.5212047100067139,
 0.5124233961105347,
 0.5239564776420593,
 0.6102941036224365,
 0.493325799703598,
 0.5247918963432312,
 0.519212543964386,
 0.6102941036224365,
 0.5320717096328735,
 0.4977165758609772,
 0.5197939276695251,
 0.5217131972312927,
 0.5380725264549255,
 0.4873267412185669,
 0.5125575065612793,
 0.516507089138031,
 0.5079290866851807,
 0.518136203289032,
 0.512977123260498,
 0.5076610445976257,
 0.5139845609664917,
 0.5508511066436768,
 0.5141720175743103,
 0.5150411128997803,
 0.5047395825386047,
 0.5151817798614502,
 0.5219705104827881]

In [30]:
test_mae

[0.3014705777168274,
 0.4161497950553894,
 0.3815137445926666,
 0.38770732283592224,
 0.4199019968509674,
 0.3967743515968323,
 0.3014705777168274,
 0.4030305743217468,
 0.4108468294143677,
 0.4169695973396301,
 0.3014705777168274,
 0.39230799674987793,
 0.40773293375968933,
 0.4089895784854889,
 0.3771910071372986,
 0.38339918851852417,
 0.37718722224235535,
 0.41205906867980957,
 0.40483003854751587,
 0.4013218283653259,
 0.40204501152038574,
 0.3977200388908386,
 0.411958783864975,
 0.3896588087081909,
 0.39495205879211426,
 0.40439867973327637,
 0.3926684856414795,
 0.3782082498073578,
 0.42153993248939514,
 0.374688982963562]

In [31]:
test_ARE

[0.1985294,
 0.1672794,
 0.1672794,
 0.17218138,
 0.1672794,
 0.17463236,
 0.1985294,
 0.17463236,
 0.17463236,
 0.1672794,
 0.1985294,
 0.15992647,
 0.1672794,
 0.18198529,
 0.1672794,
 0.17463236,
 0.15992647,
 0.17953432,
 0.17463236,
 0.17463236,
 0.17463236,
 0.1672794,
 0.15992647,
 0.15012255,
 0.17463236,
 0.16482843,
 0.17953432,
 0.17218138,
 0.1672794,
 0.1574755]

In [32]:
total_scores

[[0.6102941036224365,
  0.6102941036224365,
  0.3014705777168274,
  0.7812132239341736,
  0.16906055808067322],
 [0.49952900409698486,
  0.49952900409698486,
  0.4161497950553894,
  0.7067736387252808,
  0.13517343997955322],
 [0.5408803820610046,
  0.5408803820610046,
  0.3815137445926666,
  0.7354456782341003,
  0.14659483730793],
 [0.5212047100067139,
  0.5212047100067139,
  0.38770732283592224,
  0.7219451069831848,
  0.14319577813148499],
 [0.5124233961105347,
  0.5124233961105347,
  0.4199019968509674,
  0.7158375382423401,
  0.14029569923877716],
 [0.5239564776420593,
  0.5239564776420593,
  0.3967743515968323,
  0.7238484025001526,
  0.1426088809967041],
 [0.6102941036224365,
  0.6102941036224365,
  0.3014705777168274,
  0.7812132239341736,
  0.16906055808067322],
 [0.493325799703598,
  0.493325799703598,
  0.4030305743217468,
  0.7023715376853943,
  0.13197332620620728],
 [0.5247918963432312,
  0.5247918963432312,
  0.4108468294143677,
  0.724425196647644,
  0.1447775214910507

In [33]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.10014766827225685
median testing mse: 0.5173216462135315


In [34]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.17548320442438126
median testing mae: 0.39724719524383545


In [35]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.1721813827753067


In [36]:
accuracies = []
from sklearn.metrics import accuracy_score
for i in range(30):
    accuracies.append(accuracy_score(y, total_predictions[i]))

In [37]:
print("median accuracy is: " + str(statistics.median(accuracies)))

median accuracy is: 0.7095588235294118
